# Tutorial 5: Everything Is a Passage

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-pure-passage-calculus/blob/main/notebooks/tutorial_05_everything_is_a_passage.ipynb)

---

## Mund's Radical Insight

In Year 738, Kelleth Mund made a discovery that astonished even his supporters:

> *"I have found that the natural numbers themselves can be encoded as pure passages. The number three IS the passage that applies something three times. There is no number beyond this application."*

This tutorial explores **Church encodings**—the representation of data as pure passages.

---

## Learning Objectives

By the end of this tutorial, you will:
1. Encode **booleans** as passages
2. Encode **natural numbers** as passages (Church numerals)
3. Build **arithmetic operations** using passages
4. Encode **pairs and lists** as passages

## Setup

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

encodings = pd.read_csv(BASE_URL + "church_encodings.csv")

print(f"Loaded {len(encodings)} Church encodings")

## Part 1: Church Booleans

How can TRUE and FALSE be passages?

The key insight: **booleans are selectors**.
- TRUE selects the first of two options
- FALSE selects the second

```
TRUE  = λx.λy.x  (select first)
FALSE = λx.λy.y  (select second)
```

In [ ]:
# Church booleans
TRUE = lambda x: lambda y: x
FALSE = lambda x: lambda y: y

# Test: TRUE selects first, FALSE selects second
print(f"TRUE('yes')('no') = {TRUE('yes')('no')}")
print(f"FALSE('yes')('no') = {FALSE('yes')('no')}")

In [ ]:
# Look up in our data
bool_encodings = encodings[encodings['encoding_type'] == 'boolean']
bool_encodings[['encoded_value', 'lambda_notation', 'notes']]

In [ ]:
# IF-THEN-ELSE is trivial with Church booleans!
# IF condition THEN a ELSE b = condition a b

IF = lambda cond: lambda then_val: lambda else_val: cond(then_val)(else_val)

print(IF(TRUE)('then')('else'))
print(IF(FALSE)('then')('else'))

In [ ]:
# Boolean operations

# AND: if p then q else FALSE
AND = lambda p: lambda q: p(q)(p)

# OR: if p then TRUE else q  
OR = lambda p: lambda q: p(p)(q)

# NOT: swap the arguments
NOT = lambda p: lambda a: lambda b: p(b)(a)

# Test
def to_bool(church_bool):
    return church_bool(True)(False)

print(f"AND TRUE TRUE = {to_bool(AND(TRUE)(TRUE))}")
print(f"AND TRUE FALSE = {to_bool(AND(TRUE)(FALSE))}")
print(f"OR FALSE TRUE = {to_bool(OR(FALSE)(TRUE))}")
print(f"NOT TRUE = {to_bool(NOT(TRUE))}")

## Part 2: Church Numerals

Numbers as passages: **n IS the act of doing something n times**.

```
0 = λf.λx.x           (apply f zero times)
1 = λf.λx.f x         (apply f once)
2 = λf.λx.f (f x)     (apply f twice)
3 = λf.λx.f (f (f x)) (apply f three times)
```

In [ ]:
# Church numerals
ZERO = lambda f: lambda x: x
ONE = lambda f: lambda x: f(x)
TWO = lambda f: lambda x: f(f(x))
THREE = lambda f: lambda x: f(f(f(x)))

# Convert to Python int for display
def to_int(n):
    return n(lambda x: x + 1)(0)

print(f"ZERO = {to_int(ZERO)}")
print(f"ONE = {to_int(ONE)}")
print(f"TWO = {to_int(TWO)}")
print(f"THREE = {to_int(THREE)}")

In [ ]:
# Look up Church numerals in our data
numeral_encodings = encodings[encodings['encoding_type'] == 'numeral']
numeral_encodings[['encoded_value', 'lambda_notation', 'notes']]

### Arithmetic Operations

In [ ]:
# SUCCESSOR: add one more application of f
SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

FOUR = SUCC(THREE)
FIVE = SUCC(FOUR)

print(f"SUCC THREE = {to_int(SUCC(THREE))}")
print(f"SUCC SUCC THREE = {to_int(SUCC(SUCC(THREE)))}")

In [ ]:
# ADDITION: m + n applications
# Apply f m times, then n more times
PLUS = lambda m: lambda n: lambda f: lambda x: m(f)(n(f)(x))

print(f"2 + 3 = {to_int(PLUS(TWO)(THREE))}")
print(f"1 + 1 = {to_int(PLUS(ONE)(ONE))}")

In [ ]:
# MULTIPLICATION: m * n applications
# Apply (n applications of f) m times
MULT = lambda m: lambda n: lambda f: m(n(f))

print(f"2 * 3 = {to_int(MULT(TWO)(THREE))}")
print(f"3 * 3 = {to_int(MULT(THREE)(THREE))}")

In [ ]:
# EXPONENTIATION: m^n
# n applications of m
EXP = lambda m: lambda n: n(m)

print(f"2^3 = {to_int(EXP(TWO)(THREE))}")
print(f"3^2 = {to_int(EXP(THREE)(TWO))}")

In [ ]:
# Look up arithmetic in our data
arith_encodings = encodings[encodings['encoding_type'] == 'arithmetic']
arith_encodings[['encoded_value', 'lambda_notation', 'notes']]

### Comparison: Is Zero?

In [ ]:
# ISZERO: n applications of "return FALSE" to TRUE
# 0 applications leaves TRUE, any other returns FALSE
ISZERO = lambda n: n(lambda x: FALSE)(TRUE)

print(f"ISZERO ZERO = {to_bool(ISZERO(ZERO))}")
print(f"ISZERO ONE = {to_bool(ISZERO(ONE))}")
print(f"ISZERO THREE = {to_bool(ISZERO(THREE))}")

## Part 3: Pairs

A **pair** is a passage that holds two values and applies a selector to them.

```
PAIR = λx.λy.λf.f x y
FST = λp.p TRUE       (select first)
SND = λp.p FALSE      (select second)
```

In [ ]:
# Pairs
PAIR = lambda x: lambda y: lambda f: f(x)(y)
FST = lambda p: p(TRUE)
SND = lambda p: p(FALSE)

# Test
my_pair = PAIR('first')('second')
print(f"FST (PAIR 'first' 'second') = {FST(my_pair)}")
print(f"SND (PAIR 'first' 'second') = {SND(my_pair)}")

In [ ]:
# Pairs of Church numerals
num_pair = PAIR(TWO)(THREE)
print(f"FST = {to_int(FST(num_pair))}")
print(f"SND = {to_int(SND(num_pair))}")

In [ ]:
# Look up pairs in our data
pair_encodings = encodings[encodings['encoding_type'] == 'pair']
pair_encodings[['encoded_value', 'lambda_notation', 'notes']]

### The PREDECESSOR Function

Subtraction is harder than addition! The predecessor of n is (n-1).

Mund's insight: use pairs to build a "shift" operation.

```
Start with (0, 0)
Apply shift n times: (0,0) → (0,1) → (1,2) → (2,3) → ...
After n shifts, the first element is n-1
```

In [ ]:
# PREDECESSOR using pairs
# Shift: (a, b) → (b, b+1)
SHIFT = lambda p: PAIR(SND(p))(SUCC(SND(p)))

# PRED: apply SHIFT n times to (0,0), take first
PRED = lambda n: FST(n(SHIFT)(PAIR(ZERO)(ZERO)))

print(f"PRED THREE = {to_int(PRED(THREE))}")
print(f"PRED TWO = {to_int(PRED(TWO))}")
print(f"PRED ONE = {to_int(PRED(ONE))}")
print(f"PRED ZERO = {to_int(PRED(ZERO))}")  # 0 - 1 = 0 (floors at 0)

## Part 4: Lists

Lists can be encoded similarly to pairs, using a fold/reduce pattern.

In [ ]:
# Church-encoded lists
# A list is a function that folds over its elements

NIL = lambda c: lambda n: n  # Empty list returns nil case
CONS = lambda h: lambda t: lambda c: lambda n: c(h)(t(c)(n))  # Prepend

# HEAD and TAIL are trickier - let's use a simpler encoding
# Scott-encoded lists (simpler for our purposes)
NIL2 = lambda on_nil: lambda on_cons: on_nil
CONS2 = lambda h: lambda t: lambda on_nil: lambda on_cons: on_cons(h)(t)

HEAD2 = lambda l: l(None)(lambda h: lambda t: h)
TAIL2 = lambda l: l(None)(lambda h: lambda t: t)
ISNIL2 = lambda l: l(TRUE)(lambda h: lambda t: FALSE)

# Test
my_list = CONS2(1)(CONS2(2)(CONS2(3)(NIL2)))
print(f"HEAD = {HEAD2(my_list)}")
print(f"HEAD of TAIL = {HEAD2(TAIL2(my_list))}")
print(f"ISNIL empty = {to_bool(ISNIL2(NIL2))}")
print(f"ISNIL non-empty = {to_bool(ISNIL2(my_list))}")

In [ ]:
# Look up list encodings
list_encodings = encodings[encodings['encoding_type'] == 'list']
list_encodings[['encoded_value', 'lambda_notation', 'notes']]

## Part 5: Summary of Encodings

In [ ]:
# Overview of all encoding types
encodings['encoding_type'].value_counts()

In [ ]:
# Timeline of discoveries
encodings.groupby('year_discovered').size().plot(kind='bar', figsize=(10, 4), color='steelblue')
import matplotlib.pyplot as plt
plt.title('Church Encodings Discovered by Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# Who discovered what?
encodings.groupby('discovered_by')['encoding_type'].value_counts()

## Exercises

### Exercise 1: Boolean Operations

Implement XOR and NAND using Church booleans:

In [ ]:
# Exercise 1 workspace

# XOR: true if exactly one is true
# Hint: XOR p q = if p then (not q) else q
XOR = None  # Your code here

# NAND: not (p and q)
NAND = None  # Your code here

# Test (uncomment when ready)
# print(f"XOR TRUE FALSE = {to_bool(XOR(TRUE)(FALSE))}")
# print(f"XOR TRUE TRUE = {to_bool(XOR(TRUE)(TRUE))}")
# print(f"NAND TRUE TRUE = {to_bool(NAND(TRUE)(TRUE))}")

### Exercise 2: Numeric Operations

Implement subtraction and comparison:

In [ ]:
# Exercise 2 workspace

# SUB: m - n (floors at 0)
# Hint: apply PRED n times to m
SUB = lambda m: lambda n: n(PRED)(m)

# LEQ: m <= n
# Hint: m - n = 0 iff m <= n
LEQ = None  # Your code here

# Test
print(f"5 - 3 = {to_int(SUB(FIVE)(THREE))}")
print(f"2 - 3 = {to_int(SUB(TWO)(THREE))}")

# Uncomment when LEQ is implemented
# print(f"2 <= 3 = {to_bool(LEQ(TWO)(THREE))}")
# print(f"3 <= 2 = {to_bool(LEQ(THREE)(TWO))}")

### Exercise 3: Pair Operations

Build a triple (3-element tuple) using pairs:

In [ ]:
# Exercise 3 workspace

# A triple (a, b, c) can be encoded as a pair of (pair of a,b, c)
# TRIPLE a b c = PAIR (PAIR a b) c

TRIPLE = lambda a: lambda b: lambda c: PAIR(PAIR(a)(b))(c)
GET1 = None  # Your code here - extract first element
GET2 = None  # Your code here - extract second element  
GET3 = None  # Your code here - extract third element

# Test (uncomment when ready)
# my_triple = TRIPLE('a')('b')('c')
# print(f"GET1 = {GET1(my_triple)}")
# print(f"GET2 = {GET2(my_triple)}")
# print(f"GET3 = {GET3(my_triple)}")

## Summary

In this tutorial, we learned:

1. **Booleans** are selectors: TRUE picks first, FALSE picks second
2. **Numbers** are iterators: n IS the act of applying f n times
3. **Arithmetic** emerges from function composition
4. **Pairs** store two values and apply selectors to them
5. **Lists** fold over their elements

Mund's insight was profound: **there are no primitive data types**. Everything—numbers, booleans, data structures—is just passages operating on passages.

### Next Tutorial

In Tutorial 6, we'll confront the dark side of this power: **self-reference and dissolution**. What happens when passages can refer to themselves? We'll meet the Y combinator and understand why Omega never terminates.